### In this hands on you will build a model that once trained on a peice of text data can generate its own sequnce of words in a similar fashion as in trained data.

- Follow the instructions provided for each cell and and code accordingly. 
- In order to run the cell press shift+enter.
- make sure you have run all the cells before submitting the hands on

### Run the below cell to import necessary packages

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

/home/shreyasrbhat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Read the text data from story.txt file and split the text into seperate tokens

### Expected output:
    array(['long', 'ago', ',', 'the', 'mice', 'had', 'a', 'general',
       'council', 'to']

In [5]:
### Start code here
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content
training_data = read_data("story.txt")
###End code
training_data[:10]

array(['long', 'ago', ',', 'the', 'mice', 'had', 'a', 'general',
       'council', 'to'], dtype='<U13')

### Take the unique tokens in training_data nas sort them alphabetical order and assign the sorted list to variable words
### create dictionary ind_to_word to map index to word
### create another dictionary word_to_ind to reverse map word to their respective index.

In [33]:
####Start code here
words = sorted(list(set(training_data)))
ind_to_word = {n:word for n, word in enumerate(words)}
word_to_ind = {word:n for n, word in enumerate(words)}
###End code
print("words: ", words[:10], "\n")
print("index_to_words: ", list(ind_to_word.items())[:10], "\n")
print("word_to_index: ", list(word_to_ind.items())[:10], "\n")

words:  [',', '.', '?', 'a', 'about', 'ago', 'agree', 'all', 'always', 'an'] 

index_to_words:  [(0, ','), (1, '.'), (2, '?'), (3, 'a'), (4, 'about'), (5, 'ago'), (6, 'agree'), (7, 'all'), (8, 'always'), (9, 'an')] 

word_to_index:  [(',', 0), ('.', 1), ('?', 2), ('a', 3), ('about', 4), ('ago', 5), ('agree', 6), ('all', 7), ('always', 8), ('an', 9)] 



### write a function to generate training dataset 
    - parameters: dataset: orginal dataset
                  look_back: the window size that tells the number of previous values in the series to look for to                   predict the next one.
    - returns: feature and target arrays
    
example: 
         for window size 1:
         dataset = [1,2,3,4]  
         feature = [[1],[2],[3]]    
         target = [2,3,4]  
         
         for window size 2:
         feature = [[1,2],[2,3]]  
         target = [3,4]  
### Expected output when you when you call generate_dataset on training_data and look_back = 10 :
input:  [[48, 5, 0, 85, 56, 37, 3, 35, 28, 92], [5, 0, 85, 56, 37, 3, 35, 28, 92, 25], [0, 85, 56, 37, 3, 35, 28, 92, 25, 102]]  
labels:  [25, 102, 53]


In [29]:
def generate_dataset(dataset, look_back=10):

    for i in range(0, len(dataset) - look_back, 1):
        sequence = dataset[i:i+look_back]
        label = dataset[i+look_back]
        inputs.append([word_to_ind[word] for word in sequence])
        labels.append(word_to_ind[label])

    return inputs, labels
inputs, labels = generate_dataset(training_data, 10)
print("input: ", inputs[:3])
print("labels: ", labels[:3])

input:  [[48, 5, 0, 85, 56, 37, 3, 35, 28, 92], [5, 0, 85, 56, 37, 3, 35, 28, 92, 25], [0, 85, 56, 37, 3, 35, 28, 92, 25, 102]]
labels:  [25, 102, 53]


### reshape the input data into (num_samples, look_back , 1)

In [35]:
X_modified = np.reshape(inputs, (len(inputs), seq_length, 1))
X_modified = X_modified / float(len(words))
Y_modified = np_utils.to_categorical(labels)

### Using keras Sequential() class create a model having two LSTM block and one fully connected  layer with activation softmax
### apply dropout with probability p = 0.2 between the layers of LSTM
### compile the model with categorical_crossentropy loss and  adam optimizer

### Expected output
<img src="lstm.png">

In [42]:
np.random.seed(51)
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 400)           643200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 400)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 400)               1281600   
_________________________________________________________________
dropout_4 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 112)               44912     
Total params: 1,969,712
Trainable params: 1,969,712
Non-trainable params: 0
_________________________________________________________________


### Run model.fit() on train data with features as X_modified and target as Y_modified for 30 epoches and batch_size = 10

In [45]:
train_logs = model.fit(X_modified, Y_modified, epochs=60,batch_size=50)
with open("output.txt", "wr") as file:
    file.write("train score {0:.2f}\n".format(trainScore))

Epoch 1/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0735
Epoch 2/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0622
Epoch 3/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0503
Epoch 4/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0408
Epoch 5/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0561
Epoch 6/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0267
Epoch 7/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0447
Epoch 8/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.2962
Epoch 9/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0841
Epoch 10/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0333
Epoch 11/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0211
Epoch 12/60
1552/1552 [==============================] - 6s 4ms/step - loss: 0.0171
E

ValueError: must have exactly one of create/read/write/append mode

### The below codes takes a random sequence of words and generates more sequnce using the model you trained above.

In [183]:
string_mapped = inputs[50].copy()
full_string = [ind_to_word[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(words))
    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [ind_to_word[value] for value in string_mapped]
    full_string.append(ind_to_word[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

    
txt=""
for word in full_string:
    txt = txt+" "+word
print(txt)

In [44]:
train_logs.history["loss"][-1]

0.15256281844556294

In [ ]:
0.00026378981566108214
